In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', 10000)
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
import matplotlib.pyplot as plt
import os
import transformers
from transformers import BertForTokenClassification, AdamW
from seqeval.metrics import f1_score, accuracy_score
import Levenshtein
import string
import difflib

transformers.__version__

torch.__version__

'1.8.1'

In [3]:
tag_values = ['O', 'PER', 'LOC', 'ORG']
#tag_values = ['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_whole_word_mask=True)
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx), output_attentions = False, output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [4]:
#model.load_state_dict(torch.load("../model_2/bert_base_conll_50.pt", map_location=torch.device('cpu'))) 
model.load_state_dict(torch.load("../model/with_punctuation_with_all_broken_entities_vertical_concatenation_100_2.pt", map_location=torch.device('cpu'))) # WITH PHONETIC NOISE
#model.load_state_dict(torch.load("../model_2/bert_base_conll_with_punctuation_with_broken_entities_75.pt", map_location=torch.device('cpu'))) # WITH BROKEN ENTITIES

<All keys matched successfully>

In [5]:
def prepare_data_for_test(filepath):
    df = pd.read_csv(filepath, sep=";")
    #df.drop(['Unnamed: 0'], axis=1, inplace=True)
    #df = df[:6723]
    g_test = df.groupby("Sentence #")
    test_df = pd.DataFrame({"Sentence": g_test.apply(lambda sdf: " ".join(sdf.Word)),
                       "Tag": g_test.apply(lambda sdf: ",".join(sdf.Tag))})
    test_df.reset_index(inplace=True)
    return df, test_df

In [6]:
def model_test(data, tokenizer, model):
    test = []
    #results = open("conll03_base_ljspeech_asr_test_without_gpe_uncased_results_lower.txt", "a+")
    #test_data=original_data['sentence'].values.tolist()
    #test_data=original_sentence
    #test_data=test_df['Sentence'].values.tolist()
    test_data=data

    # ASR TEST DATE LATEST
    sentence_no = 0
    for data in test_data:
        tokenized_sentence = tokenizer.encode(data.lower().strip())
        #tokenized_sentence = nlp(data.lower().strip())
        input_ids = torch.tensor([tokenized_sentence])
        #input_ids = torch.tensor([tokenized_sentence._.trf_word_pieces])

        with torch.no_grad():
             output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

        # join bpe split tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        #tokens = _.trf_word_pieces_
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(tag_values[label_idx])
                new_tokens.append(token)

        for token, label in zip(new_tokens, new_labels):
            #result = str(sentence_no) + "\t" + label + "\t" + token + "\n"
            #results.write(result)
            test.append((str(sentence_no), label, token))
        sentence_no = sentence_no + 1
    test_df = pd.DataFrame(test, columns=['sentence_no', 'labels', 'token'])
    return test_df
    #test_df.to_csv("final_asr_test_dataframe.csv", index=False)

In [7]:
def prepare_model_output(test_df, df):
    indexNames = test_df[test_df['token'] == "[CLS]" ].index
    test_df.drop(indexNames, inplace=True)
    indexNames = test_df[test_df['token'] == "[SEP]" ].index
    test_df.drop(indexNames, inplace=True)
    test_df.reset_index(drop=True, inplace=True)
    test_df['label_asr'] = df['Tag']
    test_df['token_asr'] = df['Word']
    return test_df

In [8]:
def statistics(test_df, tags):
    new_acc = accuracy_score(test_df['labels'].values.tolist(), test_df['label_asr'].values.tolist())
    print(new_acc)

    new_f1 = f1_score(test_df['labels'].values.tolist(), test_df['label_asr'].values.tolist())
    print(new_f1)
    print("---STATISTICS ON EACH LABEL---")
    for tag in tags:
        true_positive = test_df[((test_df['labels'].str.contains(tag)) & (test_df['label_asr'].str.contains(tag)))]
        print(len(true_positive))
        false_positive = test_df[((test_df['labels'].str.contains(tag)) & (~test_df['label_asr'].str.contains(tag)))]
        print(len(false_positive))
        false_negative = test_df[((~test_df['labels'].str.contains(tag)) & (test_df['label_asr'].str.contains(tag)))]
        print(len(false_negative))
        true_negative = test_df[((~test_df['labels'].str.contains(tag)) & (~test_df['label_asr'].str.contains(tag)))]
        print(len(true_negative))
        prec = len(true_positive) / (len(true_positive) + len(false_positive))
        print(prec)
        recall = len(true_positive) / (len(true_positive) + len(false_negative))
        print(recall)
        f_measure = (2 * prec * recall) / (prec + recall)
        print(f_measure)
        print("---------------------------------------")

In [9]:
df, test_df = prepare_data_for_test('unprocessed_sampled_asr2.csv')

In [10]:
test_df.head()

,Sentence #,Sentence,Tag
0,2.0,for although the Chinese took Impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands by a similar process,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,LOC,O,O,O,O"
1,23.0,what the first Bible actually dated which also was printed at mace by Peter Shaffer in the year 1462,"O,O,O,O,O,O,O,O,O,O,O,LOC,O,PER,PER,O,O,O,O"
2,26.0,especially as regards to lowercase letters and type very similar was used during the next 15 or 20 years not only by chauffeur,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,PER"
3,27.0,Buy printers in Strasburg basil Paris Lubec and other cities,"O,O,O,LOC,O,LOC,LOC,O,O,O"
4,28.0,but don ' t expect in Italy letter with most often used,"O,O,O,O,O,O,LOC,O,O,O,O,O"


In [11]:
test_df = model_test(test_df['Sentence'].values.tolist(), tokenizer, model)

In [12]:
test_df = prepare_model_output(test_df, df)

In [13]:
test_df.tail()

,sentence_no,labels,token,label_asr,token_asr
8749,472,O,called,O,called
8750,472,O,the,O,The
8751,472,ORG,cellar,ORG,Cellar
8752,472,ORG,coffee,ORG,Coffee
8753,472,O,house,O,House


In [14]:
test_df['label_asr'].unique()

array(['O', 'LOC', 'PER', 'ORG'], dtype=object)

In [15]:
g_test = test_df.groupby("sentence_no")
test = pd.DataFrame({"model_tag": g_test.apply(lambda sdf: sdf.labels.values.tolist()),
                       "asr_tag": g_test.apply(lambda sdf: sdf.label_asr.values.tolist())})

In [16]:
test.head()

,model_tag,asr_tag
sentence_no,,
0,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, LOC, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, LOC, O, O, O, O]"
1,"[O, O, O, O, O, O, O, O, O, O, O, LOC, O, PER, PER, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, LOC, O, PER, PER, O, O, O, O]"
10,"[O, O, LOC, O, O, O, O, O, O, O, O, LOC, O, O, O, O]","[O, O, LOC, O, O, O, O, O, O, O, O, LOC, O, O, O, O]"
100,"[O, O, O, O, O, O, O, O, O, PER, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
101,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, PER, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, PER, O, O, O, O]"


In [17]:
test['asr_sentence_no'] = test.index
test[["asr_sentence_no"]] = test[["asr_sentence_no"]].apply(pd.to_numeric)
test.sort_values('asr_sentence_no', inplace=True)
test.reset_index(drop=True, inplace=True)

In [18]:
test.head()

,model_tag,asr_tag,asr_sentence_no
0,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, LOC, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, LOC, O, O, O, O]",0
1,"[O, O, O, O, O, O, O, O, O, O, O, LOC, O, PER, PER, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, LOC, O, PER, PER, O, O, O, O]",1
2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, PER]",2
3,"[O, O, O, LOC, ORG, LOC, ORG, O, O, O]","[O, O, O, LOC, O, LOC, LOC, O, O, O]",3
4,"[O, O, O, O, O, O, LOC, O, O, O, O, O]","[O, O, O, O, O, O, LOC, O, O, O, O, O]",4


In [19]:
print("Accuracy: " , accuracy_score(test['model_tag'].values.tolist(), test['asr_tag'].values.tolist()))
print("F1 Score: ",f1_score(test['model_tag'].values.tolist(), test['asr_tag'].values.tolist()))
#statistics(test_df, ['PER', 'ORG', 'LOC', 'O'])
#0.7758389261744967 without punctuation
#0.676056338028169 with punctuation 1

Accuracy:  0.9710989262051634
F1 Score:  0.8881469115191986


/Users/shahzainmehboob/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/shahzainmehboob/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/shahzainmehboob/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [20]:
def prepare_data_for_analysis(test_df, original_data_path):
    g_asr = test_df.groupby("sentence_no")
    asr_df = pd.DataFrame({'Sentence': g_asr.apply(lambda sdf: " ".join(map(str,sdf.token))),
                      'Tag': g_asr.apply(lambda sdf: ",".join(sdf.labels))})
    asr_df['asr_sentence_no'] = asr_df.index
    asr_df[["asr_sentence_no"]] = asr_df[["asr_sentence_no"]].apply(pd.to_numeric)
    asr_df.sort_values('asr_sentence_no', inplace=True)
    asr_df.reset_index(drop=True, inplace=True)
    original = pd.read_csv(original_data_path, sep=";")
    #original.drop(['Unnamed: 0'], axis=1, inplace=True)
    #original = original[:6723]
    g_original = original.groupby("Sentence #")
    original_df = pd.DataFrame({'Sentence': g_original.apply(lambda sdf: " ".join(map(str,sdf.Word))),
                      'Tag': g_original.apply(lambda sdf: ",".join(sdf.Tag))})
    original_df.reset_index(inplace=True)
    combined_df = pd.DataFrame({"original_sentence": original_df['Sentence'].str.lower(),
                           "original_tags": original_df['Tag'], 
                           "asr_sentence": asr_df['Sentence'],
                           "asr_tags": asr_df['Tag']})
    return asr_df, combined_df

In [21]:
def pattern_finding(tag, combined_df):
#tag = "PER"
    analysis = []
    for i in range(0, len(combined_df), 1):
        sample = combined_df.loc[[i]]
        for original_sentence, asr_sentence, original_tag, asr_tag in zip(sample['original_sentence'].values.tolist(),
                                                                          sample['asr_sentence'].values.tolist(),
                                                                          sample['original_tags'].values.tolist(),
                                                                          sample['asr_tags'].values.tolist()):
            original_tag_token = np.array(original_tag.split(","))
            asr_tag_token = np.array(asr_tag.split(","))
            original_label = np.array(original_sentence.lower().split())
            asr_label = np.array(asr_sentence.lower().split())

            if tag in original_tag_token:
                original_tag_ind = [index for index, element in enumerate(original_tag_token) if
                                    original_tag_token[index] == tag]
                if tag in asr_tag_token:
                    asr_tag_ind = [index for index, element in enumerate(asr_tag_token) if
                                       asr_tag_token[index] == tag]
                    
                    asr_tokens = []
                    original_tokens = []
                    errors = []
                        # Sweynheim pannartz
                        # Swain heim pannartz
                    for ind in original_tag_ind:
                        original_entity = original_label[ind]
                        asr_entity = difflib.get_close_matches(original_entity, asr_label[asr_tag_ind])
                        if len(asr_entity) > 0:
                            asr_entity = asr_entity[0]
                            error = (1 - (Levenshtein.distance(original_entity, asr_entity) / max(len(original_entity), len(asr_entity)))) * 100
                            if error >= 50:
                                asr_tokens.append(asr_entity)
                                original_tokens.append(original_entity)
                                errors.append(error)
                            else:
                                asr_tokens.append(asr_label[ind])
                                original_tokens.append(original_entity)
                                errors.append(0.0)
                        else:
                            asr_tokens.append(asr_label[ind])
                            original_tokens.append(original_entity)
                            errors.append(0.0)
                    analysis.append((i, original_tokens, asr_tokens, errors, np.mean(errors), True))
                else:
                    check = []
                    o_label = original_label[original_tag_ind]
                    for lab in o_label:
                        j = 0
                        for asr_lab in asr_label:
                            local_error = (1 - (Levenshtein.distance(lab, asr_lab) / max(len(lab), len(asr_lab)))) * 100
                            if local_error >= 50.0:
                                check.append(j)
                            j = j + 1
                    if len(check) > 0:
                        asr_tokens = []
                        original_tokens = []
                        errors = []
                        for ind in original_tag_ind:
                            original_entity = original_label[ind]
                            asr_entity = difflib.get_close_matches(original_entity, asr_label[check])
                            if len(asr_entity) > 0:
                                asr_entity = asr_entity[0]
                                error = (1 - (Levenshtein.distance(original_entity, asr_entity) / max(
                                len(original_entity), len(asr_entity)))) * 100
                                asr_tokens.append(asr_entity)
                                original_tokens.append(original_entity)
                                errors.append(error)
                            else:
                                asr_tokens.append(asr_label[ind])
                                original_tokens.append(original_entity)
                                errors.append(0.0)
                        analysis.append((i, original_tokens, asr_tokens, errors, np.mean(errors), False))
                    else:
                        analysis.append((i, original_label[original_tag_ind], ["None"], [0.0], 0.0, False))
    return analysis

In [22]:
asr_df, combined_df = prepare_data_for_analysis(test_df, 'unprocessed_sampled_asr2.csv')

In [37]:
analysis_df = pd.DataFrame(pattern_finding("ORG", combined_df), columns=['Sample #', 'ASR Output', 'Model Output', 'Lavenstein', 'Lavenstein Mean', 'Flag'])

In [38]:
orig_asr_found_complete = analysis_df[(analysis_df['Flag'] == True) & (analysis_df['Lavenstein Mean'] == 100.0)]
orig_asr_found_complete_per = (len(orig_asr_found_complete) / len(analysis_df)) * 100
print(orig_asr_found_complete_per)
orig_asr_found_complete.head()
print(len(orig_asr_found_complete))

46.42857142857143
39


In [39]:
orig_asr_found = analysis_df[(analysis_df['Flag'] == True) & (analysis_df['Lavenstein Mean'] < 100.0) & (analysis_df['Lavenstein Mean'] >= 0.0)]
orig_asr_found_per = (len(orig_asr_found) / len(analysis_df)) * 100
print(orig_asr_found_per)
print(len(orig_asr_found))
orig_asr_found.head()

14.285714285714285
12


,Sample #,ASR Output,Model Output,Lavenstein,Lavenstein Mean,Flag
6,96,"[kings, bench]","[kings, bench]","[100.0, 0.0]",50.000000,True
7,97,"[the, fleet]","[the, fleet]","[0.0, 100.0]",50.000000,True
14,120,"[kings, bench]","[kings, bench]","[100.0, 0.0]",50.000000,True
17,125,"[marshalsea, court]","[marshalsea, court]","[0.0, 0.0]",0.000000,True
36,254,"[prison, discipline, society]","[prison, discipline, society]","[0.0, 0.0, 100.0]",33.333333,True


In [40]:
orig_asr_similar = analysis_df[(analysis_df['Flag'] == False) & (analysis_df['Lavenstein Mean'] <= 100.0) & (analysis_df['Lavenstein Mean'] > 0.0)]
orig_asr_similar_per = (len(orig_asr_similar) / len(analysis_df)) * 100
print(orig_asr_similar_per)
orig_asr_similar.head()
print(len(orig_asr_similar))

39.285714285714285
33


In [41]:
orig_asr_nofound = analysis_df[(analysis_df['Flag'] == False) & (analysis_df['Lavenstein Mean'] <= 0.0)]
orig_asr_nofound_per = (len(orig_asr_nofound) / len(analysis_df))*100
print(orig_asr_nofound_per)
orig_asr_nofound.head()
print(len(orig_asr_nofound))

0.0
0


In [42]:
[orig_asr_found_complete_per, orig_asr_found_per, orig_asr_similar_per, orig_asr_nofound_per]
# [84.375, 3.65, 11.97] PER
# [162, 7, 23] PER count


# [84.92, 8.33, 6.746] LOC
# [214, 21, 17] LOC count


# [40.476, 13.09, 46.42] ORG
# [34, 11, 39] ORG count

# [69.92, 8.35, 21.71] Total Percentage





# [85.41666666666666, 5.208333333333334, 9.375, 0.0] PER broken
# [164, 10, 18, 0] PER broken count


# [83.73015873015873, 3.968253968253968, 12.3015873015873, 0.0] LOC broken
# [211, 10, 31, 0] LOC broken count


# [46.42857142857143, 14.285714285714285, 39.285714285714285, 0.0] ORG broken
# [39, 12, 33, 0] ORG broken count


# [71.85, 7.81, 20.31] total broken percentage


[46.42857142857143, 14.285714285714285, 39.285714285714285, 0.0]

In [43]:
[len(orig_asr_found_complete), len(orig_asr_found), len(orig_asr_similar), len(orig_asr_nofound)]

[39, 12, 33, 0]

In [ ]:
# [131, 8, 18, 0]